In [2]:
import cv2
import mediapipe as mp

In [3]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils  # Utility to draw landmarks

In [4]:
# Open webcam
cap = cv2.VideoCapture(0)

In [5]:
# Initialize Hand Detector
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process frame
        results = hands.process(frame_rgb)

        # Draw hand landmarks if detected
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Define finger structure (Tip, DIP, PIP, MCP)
                fingers = {
                    "Thumb": (4, 3, 2, 1),
                    "Index": (8, 7, 6, 5),
                    "Middle": (12, 11, 10, 9),
                    "Ring": (16, 15, 14, 13),
                    "Pinky": (20, 19, 18, 17)
                }

                h, w, _ = frame.shape
                extended_fingers = []

                for finger_name, (tip, dip, pip, mcp) in fingers.items():
                    # Get joint coordinates
                    tip_x, tip_y = int(hand_landmarks.landmark[tip].x * w), int(hand_landmarks.landmark[tip].y * h)
                    dip_x, dip_y = int(hand_landmarks.landmark[dip].x * w), int(hand_landmarks.landmark[dip].y * h)
                    pip_x, pip_y = int(hand_landmarks.landmark[pip].x * w), int(hand_landmarks.landmark[pip].y * h)
                    mcp_x, mcp_y = int(hand_landmarks.landmark[mcp].x * w), int(hand_landmarks.landmark[mcp].y * h)

                    # Draw circles for each joint
                    cv2.circle(frame, (tip_x, tip_y), 6, (0, 255, 0), -1)  # Tip
                    cv2.circle(frame, (dip_x, dip_y), 5, (255, 0, 0), -1)  # DIP
                    cv2.circle(frame, (pip_x, pip_y), 4, (0, 0, 255), -1)  # PIP
                    cv2.circle(frame, (mcp_x, mcp_y), 3, (255, 255, 0), -1)  # MCP

                    # Display joint names
                    cv2.putText(frame, f"{finger_name} Tip", (tip_x + 10, tip_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                    cv2.putText(frame, f"DIP", (dip_x + 10, dip_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)
                    cv2.putText(frame, f"PIP", (pip_x + 10, pip_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
                    cv2.putText(frame, f"MCP", (mcp_x + 10, mcp_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1)

                    # Check if finger is extended
                    if tip_y < pip_y:  # Tip above PIP joint
                        extended_fingers.append(finger_name)

                # Gesture Recognition
                if extended_fingers == ["Thumb"]:
                    cv2.putText(frame, "👍 Thumbs Up", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)
                elif extended_fingers == ["Index", "Middle"]:
                    cv2.putText(frame, "✌ Victory Sign", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 3)
                elif extended_fingers == ["Index", "Pinky", "Thumb"]:
                    cv2.putText(frame, "🤘 Rock Sign", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 3)

        # Display output
        cv2.imshow("Hand Gesture & Joint Detection", frame)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

KeyboardInterrupt: 

In [ ]:
# Release resources
cap.release()
cv2.destroyAllWindows()